# Statistics on Power Outages in the U.S from January 2000 to July 2016

#### Mohamed Elshazly

### Dataset from here: https://engineering.purdue.edu/LASCI/research-data/outages

### Corresponding website: 

 This dataset covers major power outages in the entire United States from 2000 to 2016. The following aims to seek answers to the relationship between the state that one resides, and the likelihood of experiencing a major power outage.

In [125]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
pd.options.plotting.backend = 'plotly'

## Cleaning and EDA

In [126]:
excel_read = pd.read_excel(os.path.join('outage.xlsx'), header = 5)
data = excel_read.copy()
data = data.iloc[1:, 2:]
population_col = data['POPULATION']
data = data.loc[:, "YEAR": "CUSTOMERS.AFFECTED"]
data['OUTAGE.START'] = pd.to_datetime(data['OUTAGE.START.DATE'].astype(str))
data['OUTAGE.RESTORATION'] = pd.to_datetime(data['OUTAGE.RESTORATION.DATE'].astype(str))
data['POPULATION'] = population_col
data = data.drop(columns = ['OUTAGE.START.DATE', 'OUTAGE.RESTORATION.DATE'])

In [127]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.TIME,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,OUTAGE.START,OUTAGE.RESTORATION,POPULATION
1,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,17:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,2011-07-01,2011-07-03,5348119.0
2,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,18:38:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,2014-05-11,2014-05-11,5457125.0
3,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,20:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,2010-10-26,2010-10-28,5310903.0
4,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,04:30:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,2012-06-19,2012-06-20,5380443.0
5,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,02:00:00,07:00:00,severe weather,NaN,NaN,1740,250,250000.0,2015-07-18,2015-07-19,5489594.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,08:00:00,20:00:00,public appeal,NaN,NaN,720,155,34500.0,2011-12-06,2011-12-06,685326.0
1531,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650,NaN,NaT,NaT,649422.0
1532,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,22:54:00,23:53:00,islanding,NaN,NaN,59,84,NaN,2009-08-29,2009-08-29,807067.0
1533,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,11:00:00,14:01:00,islanding,NaN,NaN,181,373,NaN,2009-08-29,2009-08-29,807067.0


## Plots

In [128]:
data_by_customer = data.groupby('YEAR')['U.S._STATE'].count()
plot_one = px.line(data_by_customer, x = data_by_customer.index, y = 'U.S._STATE',
                  labels = {'U.S._STATE' : 'Frequency of Outages',
                           'YEAR' : 'Year'},
                  title = 'Number of Power Outages 2000 - 2016')
plot_one

In [129]:
data_by_severity = data.groupby('YEAR')['OUTAGE.DURATION'].sum()
plot_two = px.line(data_by_severity, x = data_by_severity.index, y = 'OUTAGE.DURATION',
                  labels = {'OUTAGE.DURATION' : "Lengths of Outages (in minutes)",
                           'YEAR' : 'Year'},
                   title = 'Total Duration of Power Outages 2000 - 2016')
plot_two

In [130]:
data_by_severity_ca = data.groupby('YEAR')['CUSTOMERS.AFFECTED'].sum()
plot_three = px.line(data_by_severity_ca, x = data_by_severity_ca.index, y = 'CUSTOMERS.AFFECTED',
                  labels = {'CUSTOMERS.AFFECTED' : "Number of Customers Affected",
                           'YEAR' : 'Year'},
                   title = 'Total Customers Affected by Power Outages 2000 - 2016')
plot_three.write_html("assets/plot3.html")
plot_three

In [131]:
scatter_anomaly_duration = px.scatter(data, x = 'ANOMALY.LEVEL', y = 'OUTAGE.DURATION',
                                     labels = {"OUTAGE.DURATION" : "Duration of Power Outage", 
                                              "ANOMALY.LEVEL" : "Anomaly Level"},
                                     title = "Duration of Power Outages by Anomaly")
scatter_anomaly_duration.write_html("assets/scatter1.html")
scatter_anomaly_duration

In [132]:
outage_freq = data['U.S._STATE'].value_counts().reset_index()
outage_freq.columns = ['State', 'Frequency']
scatter_state_freq = px.scatter(outage_freq, 
                                x='State', 
                                y='Frequency',
                                labels={"State": "States", "Frequency": "Frequency of Power Outage"},
                                title="Frequency of Power Outages by State")


scatter_state_freq.write_html("assets/scatter2.html")
scatter_state_freq

In [133]:
data_by_cause = data.groupby(['YEAR', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL']).count()['POPULATION']
data_by_cause

YEAR    CAUSE.CATEGORY                 CAUSE.CATEGORY.DETAIL    
2000.0  equipment failure              failure                       1
                                       line fault                    1
                                       relaying malfunction          2
                                       transformer outage            1
        intentional attack             vandalism                     2
                                                                    ..
2016.0  intentional attack             vandalism                    14
        system operability disruption  public appeal                 1
                                       transmission interruption     3
                                       uncontrolled loss             4
                                       voltage reduction             1
Name: POPULATION, Length: 195, dtype: int64

## Assessment of Missingness

In [134]:
customers_null = data.assign(nullval = data['CUSTOMERS.AFFECTED'].isna())
null_category_count = customers_null.pivot_table(index = 'CAUSE.CATEGORY',
                                                columns = 'nullval',
                                                aggfunc = 'size',
                                                fill_value = 0)
null_prop = null_category_count.div(null_category_count.sum(axis=1), axis=0)
null_prop_difference = null_prop.assign(Abs_difference = abs(null_prop[True] - null_prop[False]))
observed_tvd = null_prop_difference['Abs_difference'].sum() / 2
observed_tvd

1.4147061232798652

In [135]:
null_prop_difference

nullval,False,True,Abs_difference
CAUSE.CATEGORY,,,
equipment failure,0.500000,0.500000,0.000000
fuel supply emergency,0.137255,0.862745,0.725490
intentional attack,0.476077,0.523923,0.047847
islanding,0.739130,0.260870,0.478261
public appeal,0.304348,0.695652,0.391304
severe weather,0.939712,0.060288,0.879423
system operability disruption,0.653543,0.346457,0.307087


In [ ]:
shuffled = data.copy()
simulated_cause_tvd = []
for i in range(10000):
    shuffled_null = shuffled.assign(nullval = shuffled['CUSTOMERS.AFFECTED'].isna())
    shuffled_null['nullval'] = np.random.permutation(shuffled_null['nullval'])
    shuffled_count = shuffled_null.pivot_table(index = 'CAUSE.CATEGORY',
                                                columns = 'nullval',
                                                aggfunc = 'size',
                                                fill_value = 0)
    shuffled_prop = shuffled_count.div(shuffled_count.sum(axis=1), axis=0)
    shuffled_diff = shuffled_prop.assign(Abs_difference = abs(shuffled_prop[True] - shuffled_prop[False]))
    simulated_tvd = shuffled_diff['Abs_difference'].sum() / 2
    simulated_cause_tvd.append(simulated_tvd)

In [ ]:
figure = px.histogram(pd.DataFrame(simulated_cause_tvd), x = 0, nbins = 5, histnorm = 'probability',
                     labels = {"0" : "Total Variation Distance"},
                     title = "Missingness of Customers Affected by Cause Category")
p_val_cause = (simulated_cause_tvd > observed_tvd).sum() / 10000
print(p_val_cause)

figure.add_vline(x = observed_tvd, line_color = 'red')
figure.write_html("assets/plot-missing1.html")
figure

# change text to "This is the histogram showing the probability of getting a total variation distance after shuffling the affected missing column. The red line indicates the observed total variation distance from the dataset. We can say then that the missingness of values in the customers affected column may not depend on the cause of the power outage." 

In [ ]:
null_climate_count = customers_null.pivot_table(index = 'CLIMATE.CATEGORY',
                                                columns = 'nullval',
                                                aggfunc = 'size',
                                                fill_value = 0)
null_climate_prop = null_climate_count.div(null_climate_count.sum(axis=1), axis=0)
null_climate_prop_difference = null_climate_prop.assign(Abs_difference = abs(null_climate_prop[True] - null_climate_prop[False]))
observed_climate_tvd = null_climate_prop_difference['Abs_difference'].sum() / 2
observed_climate_tvd

In [ ]:
shuffled_two = data.copy()
simulated_climate_tvd = []
for i in range(10000):
    shuffled_two_null = shuffled_two.assign(
        nullval = shuffled_two['CUSTOMERS.AFFECTED'].isna())
    shuffled_two_null['nullval'] = np.random.permutation(shuffled_two_null['nullval'])
    shuffled_two_count = shuffled_two_null.pivot_table(index = 'CLIMATE.CATEGORY',
                                                columns = 'nullval',
                                                aggfunc = 'size',
                                                fill_value = 0)
    shuffled_two_prop = shuffled_two_count.div(shuffled_two_count.sum(axis = 1), axis = 0)
    shuffled_two_diff = shuffled_two_prop.assign(Abs_difference = abs(
        shuffled_two_prop[True] - shuffled_two_prop[False]))
    simulated_two_tvd = shuffled_two_diff['Abs_difference'].sum() / 2
    simulated_climate_tvd.append(simulated_two_tvd)

In [ ]:
figure = px.histogram(pd.DataFrame(simulated_climate_tvd), x = 0, nbins = 25, histnorm = 'probability',
                     labels = {"0" : "total variation distance"},
                     title = 'Missingness of Customers Affected by Climate Category')
p_val_climate = (simulated_climate_tvd > observed_climate_tvd).sum() / 10000

print(p_val_climate)

figure.add_vline(x = observed_climate_tvd, line_color = 'red')
figure.write_html("assets/plot-missing2.html")
figure

In [ ]:
bin_edges = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, data['OUTAGE.DURATION'].max()]
customers_null['DURATION.BIN'] = pd.cut(data['OUTAGE.DURATION'], bins=bin_edges, right=False)

null_duration_count = customers_null.pivot_table(index = 'DURATION.BIN',
                                                columns = 'nullval',
                                                aggfunc = 'size',
                                                fill_value = 0)

null_duration_prop = null_duration_count.div(null_duration_count.sum(axis=1), axis=0)
null_duration_prop_difference = null_duration_prop.assign(
    Abs_difference=abs(null_duration_prop[True] - null_duration_prop[False])
)
observed_duration_statistic= null_duration_prop_difference['Abs_difference'].sum() / 2
observed_duration_statistic

In [ ]:
simulated_duration_tvd = []
shuffled_three = customers_null.copy()
for i in range(10000):
    shuffled_three_null = shuffled_two.assign(nullval = 
                                              shuffled_three['CUSTOMERS.AFFECTED'].isna())
    
    shuffled_nullval = np.random.permutation(customers_null['nullval'])
    shuffled_three['shuffled_nullval'] = shuffled_nullval
    
    shuffled_count = shuffled_three.pivot_table(
        index='DURATION.BIN',
        columns='shuffled_nullval',
        aggfunc='size',
        fill_value=0
    )
    
    shuffled_prop = shuffled_count.div(shuffled_count.sum(axis=1), axis=0)
    shuffled_diff = shuffled_prop.assign(
        Abs_difference=abs(shuffled_prop[True] - shuffled_prop[False])
    )
    simulated_tvd = shuffled_diff['Abs_difference'].sum() / 2
    simulated_duration_tvd.append(simulated_tvd)


In [ ]:
figure = px.histogram(pd.DataFrame(simulated_duration_tvd), x = 0, nbins = 25, histnorm = 'probability',
                     labels = {"0" : "proportion"},
                     title = 'Missingness of Customers Affected by Outage Duration')
p_val_duration = (simulated_duration_tvd > observed_duration_statistic).sum() / 10000

print(p_val_duration)
figure.add_vline(x = observed_duration_statistic, line_color = 'red')
figure.write_html("assets/plot-missing3.html")
figure

## Hypothesis Testing

Null Hypothesis: The average percent of people affected by power outages per state is due to random chance.

Alternative Hypothesis: The average percent of people affected by power outages per state is related to location and not random chance

In [ ]:
prop_by_state = data.assign(PROP_AFFECTED=data['CUSTOMERS.AFFECTED'].fillna(0) / data['POPULATION'])
average_affected_by_state = prop_by_state.groupby('U.S._STATE').mean(numeric_only = True)['PROP_AFFECTED']


population_by_state = data.groupby('U.S._STATE')['POPULATION'].mean()
expected_proportion = population_by_state / population_by_state.sum()

observed_proportion = average_affected_by_state.values
observed_ht_tvd = 0.5 * (np.sum(np.abs(observed_proportion - expected_proportion)))



In [ ]:
simulated_ht = []
for i in range(10000):
    ht_copy = prop_by_state.copy()
    ht_copy['PROP_AFFECTED'] = np.random.permutation(ht_copy['PROP_AFFECTED'])
    simulated_average = ht_copy.groupby('U.S._STATE').mean(numeric_only = True)['PROP_AFFECTED']
    simulated_tvd = 0.5 * np.sum(np.abs(simulated_average.values - expected_proportion.values))
    simulated_ht.append(simulated_tvd)


In [ ]:
p_value_ht = np.count_nonzero(np.array(simulated_ht) >= observed_ht_tvd) / 10000
p_value_ht

### Checking difference between included vs exluded outlier states.

In [ ]:
population_by_state_outliers = data.groupby('U.S._STATE').mean(numeric_only = True).drop(
['District of Columbia', 'Hawaii', 'Vermont', 'South Dakota', 'Montana'])['POPULATION']
average_affected_by_state_outliers = prop_by_state.groupby('U.S._STATE').mean(numeric_only = True).drop(
['District of Columbia', 'Hawaii', 'Vermont', 'South Dakota', 'Montana'])['PROP_AFFECTED']
expected_proportion_outliers = population_by_state_outliers / population_by_state_outliers.sum()
observed_proportion_outliers = average_affected_by_state_outliers.values
observed_ht_tvd_outliers = 0.5 * (np.sum(np.abs(observed_proportion_outliers - expected_proportion_outliers)))

simulated_ht_outlier = []
for i in range(10000):
    ht_copy = prop_by_state.copy()
    ht_copy['PROP_AFFECTED'] = np.random.permutation(ht_copy['PROP_AFFECTED'])
    simulated_average = ht_copy.groupby('U.S._STATE').mean(numeric_only = True).drop(
['District of Columbia', 'Hawaii', 'Vermont', 'South Dakota', 'Montana'])['PROP_AFFECTED']
    simulated_tvd = 0.5 * np.sum(np.abs(simulated_average.values - expected_proportion_outliers.values))
    simulated_ht_outlier.append(simulated_tvd)


In [ ]:
p_value_ht_outliers = np.count_nonzero(np.array(simulated_ht_outlier) >= observed_ht_tvd_outliers) / 10000
p_value_ht_outliers

In [ ]:
print(p_value_ht, p_value_ht_outliers)
fig = px.histogram(pd.DataFrame(simulated_ht), x = 0, nbins = 50, histnorm= 'probability',
                  title = 'Probability of Proportion Affected by Power Outage')
fig.add_vline(observed_ht_tvd, line_color = 'orange')
fig.add_vline(observed_ht_tvd_outliers, line_color = 'red')
fig.write_html("assets/plot-hypotest.html")
fig

## Reject null hypothesis in both cases

# Predicting Power Outages

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
predict_data = excel_read.copy()
predict_data = predict_data.iloc[1:, 2:]

population_col = predict_data["POPULATION"]
predict_data = predict_data.loc[:, "YEAR":"TOTAL.CUSTOMERS"]

predict_data['OUTAGE.START'] = pd.to_datetime(predict_data['OUTAGE.START.DATE'].astype(str))
predict_data['OUTAGE.RESTORATION'] = pd.to_datetime(predict_data['OUTAGE.RESTORATION.DATE'].astype(str))
predict_data['POPULATION'] = population_col

predict_data = predict_data.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
                  'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'TOTAL.PRICE', 'RES.PERCEN'], axis=1)

predict_data['CLIMATE.CATEGORY'] = predict_data['CLIMATE.CATEGORY'].fillna(predict_data['CLIMATE.CATEGORY'].mode()[0])
predict_data['CUSTOMERS.AFFECTED'] = predict_data['CUSTOMERS.AFFECTED'].fillna(predict_data['CUSTOMERS.AFFECTED'].median())
predict_data['TOTAL.SALES'] = predict_data['TOTAL.SALES'].fillna(predict_data['TOTAL.SALES'].median())


In [ ]:
predict_data

In [ ]:
X_Baseline_data = predict_data.drop('CUSTOMERS.AFFECTED', axis=1)[["COM.CUSTOMERS", "IND.CUSTOMERS"]]
y_Baseline_data = predict_data["CUSTOMERS.AFFECTED"]

In [ ]:
X_Btrain, X_Btest, y_Btrain, y_Btest = train_test_split(X_Baseline_data, y_Baseline_data)

pl = Pipeline([
    ('linreg', LinearRegression())
])

train_base_err = (-1 * cross_val_score(pl, X_Baseline_data, y_Baseline_data, cv=5, scoring='neg_root_mean_squared_error').mean())

pl.fit(X_Btrain, y_Btrain)
y_pred = pl.predict(X_Btest)
test_base_err = np.sqrt(mean_squared_error(y_Btest, y_pred))

train_base_err, test_base_err

In [ ]:
threshold_energy = predict_data['TOTAL.SALES'].median()

X_Final_data = predict_data.drop('CUSTOMERS.AFFECTED', axis=1)[["COM.CUSTOMERS", "IND.CUSTOMERS", "TOTAL.SALES", "CLIMATE.CATEGORY"]]
y_Final_data = predict_data["CUSTOMERS.AFFECTED"]

X_Ftrain, X_Ftest, y_Ftrain, y_Ftest = train_test_split(X_Final_data, y_Final_data)

preproc = ColumnTransformer(
    transformers=[
        ('energy_draw', Binarizer(threshold=threshold_energy), ['TOTAL.SALES']),
        ('cat_cols', OneHotEncoder(), ['CLIMATE.CATEGORY'])
    ],
    remainder='passthrough'
)

pl = Pipeline([
    ('preproc', preproc),
    ('linreg', LinearRegression())
])

train_final_err = (-1 * cross_val_score(pl, X_Ftrain, y_Ftrain, cv=5, scoring='neg_root_mean_squared_error').mean())

pl.fit(X_Ftrain, y_Ftrain)
y_pred = pl.predict(X_Ftest)
test_final_err = np.sqrt(mean_squared_error(y_Ftest, y_pred))


train_final_err, test_final_err


# Test to see which feature combination produces the best model.

## Baseline (# of commercial customers, # of industrial customers)
## Hybrid (# of commercial customers, # of industrial customers, climate category)
## Final (# of commercial customers, # of industrial customers, climate category, energy draw)


In [ ]:
pipes = {
    'p1': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ['COM.CUSTOMERS', 'IND.CUSTOMERS'])],
            remainder='drop')),
        ('lin-reg', LinearRegression())
    ]),
    'p2': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ["COM.CUSTOMERS", "IND.CUSTOMERS"]),
             ('ohe', OneHotEncoder(), ["CLIMATE.CATEGORY"])],
            remainder="drop")),
        ('lin-reg', LinearRegression())
    ]),
    'p3': Pipeline([
        ('trans', ColumnTransformer(
            [('keep', FunctionTransformer(lambda x: x), ["COM.CUSTOMERS", "IND.CUSTOMERS"]),
             ('ohe', OneHotEncoder(), ["CLIMATE.CATEGORY"]),
             ('binarize', Binarizer(threshold=threshold_energy), ["TOTAL.SALES"])],
            remainder="drop")),
        ('lin-reg', LinearRegression())
    ])
}

pipe_df = pd.DataFrame()
X = predict_data.drop("CUSTOMERS.AFFECTED", axis=1)
y = predict_data["CUSTOMERS.AFFECTED"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

for pipe in pipes:
    errs = cross_val_score(pipes[pipe], X_train, y_train, cv=5, scoring="neg_root_mean_squared_error")
    pipe_df[pipe] = -errs

pipe_df.index = [f"Fold {i}" for i in range(1, 6)]
pipe_df.index.name = "Validation Fold"
pipe_df.mean().idxmin()

In [ ]:
pipe_df

In [ ]:
X_Ftest['residential_draw'] = (predict_data["RES.SALES"] > (predict_data["COM.SALES"] + predict_data["IND.SALES"]))
pl.fit(X_Ftrain, y_Ftrain)
pred = pl.predict(X_Ftest)
X_Ftest["pred"] = pred
X_Ftest["real"] = y_Ftest
obs = -1 * (X_Ftest.groupby("residential_draw")
            .apply(lambda x: np.sqrt(mean_squared_error(x["real"], x["pred"]))))
obs = abs(obs[True] - obs[False])

In [ ]:
diff_in_acc = []
for _ in range(1000):
    shuffled = X_Ftest['residential_draw'].sample(frac=1.0, replace=False).reset_index(drop=True).fillna(0)
    shuffled_df = X_Ftest.copy()
    shuffled_df['residential_draw'] = shuffled
    
    shuffled_metric = -1 * (shuffled_df.groupby("residential_draw")
                            .apply(lambda x: np.sqrt(mean_squared_error(x["real"], x["pred"]))))
    shuffled_metric = shuffled_metric.reindex([True, False], fill_value=0)
    shuffled_metric = abs(shuffled_metric[True] - shuffled_metric[False])
    diff_in_acc.append(shuffled_metric)

diff_in_acc = pd.DataFrame(diff_in_acc, columns = ['Difference in MSE'])

In [ ]:
p_val_fairness = np.count_nonzero(diff_in_acc >= obs) / 10000
p_val_fairness

In [ ]:
fig = px.histogram(diff_in_acc)
fig.add_vline(x = obs, line_color = 'red')
fig.write_html("assets/plot_pred_hypotest.html")
fig